In [1]:
#In this version: 
#replaced "from deepspeed import deepspeed" with "import deepspeed" in both train.py and config.py
#Put 24 hour multivariate hourly prediction as default in run-ds.py
!git clone https://github.com/simoneliasen/query-selector.git
!pip3 install deepspeed

Cloning into 'query-selector'...
remote: Enumerating objects: 164, done.
remote: Counting objects: 100% (146/146), done.
remote: Compressing objects: 100% (134/134), done.
remote: Total 164 (delta 100), reused 28 (delta 12), pack-reused 18
Receiving objects: 100% (164/164), 12.42 MiB | 15.70 MiB/s, done.
Resolving deltas: 100% (104/104), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 665 kB 5.0 MB/s 
     |████████████████████████████████| 54 kB 2.8 MB/s 
     |████████████████████████████████| 120 kB 78.0 MB/s 
  Created wheel for deepspeed: filename=deepspeed-0.7.4-py3-none-any.whl size=676699 sha256=2088cfce10714523a473592ee7c0e5f93d3d6df42da11005eb9459ef29d396e0
  Stored in directory: /root/.cache/pip/wheels/2e/6b/f8/bf450c749abd7dccc77d0df7f2d5c4466d3288c84bdf7b8ce1
Successfully built deepspeed


In [2]:
%cd query-selector

/content/query-selector


In [10]:
#Work on subset of data for testing purposes
import pandas as pd
import numpy as np

etth1 = pd.read_csv("data/ETTh1.csv")
hub = pd.read_csv("data/dataset_dropNA.csv")

etth1['date'] = hub['date']
etth1['HUFL'] = hub['TH_NP15_GEN-APND']
etth1['HULL'] = hub['TH_SP15_GEN-APND']
etth1['MUFL'] = hub['TH_ZP26_GEN-APND']
etth1['MULL'] = hub['CAISO-SP15 Wind Power Generation Forecast']
etth1['LUFL'] = hub['CAISO-NEVP Power Demand Forecast']
etth1['LULL'] = hub['CAISO-AZPS Power Demand Forecast']
etth1['OT'] = hub['CAISO-SDGE Power Demand Forecast']

etth1.to_csv('data/ETTh1.csv', index=False)

In [9]:
#configuration of model can be configured, by configuring the "conf" variable to the correct .json
!python3 run-ds.py

{
  "data": "ETTh1",
  "seq_len": 48,
  "pred_len": 24,
  "dec_seq_len": 48,
  "hidden_size": 96,
  "heads": 2,
  "n_encoder_layers": 3,
  "encoder_attention": "query_selector_0.85",
  "n_decoder_layers": 3,
  "decoder_attention": "full",
  "batch_size": 32,
  "embedding_size": 24,
  "prediction_type": "multi",
  "dropout": 0,
  "fp16": true,
  "deepspeed": true,
  "iterations": 5,
  "exps": 2,
  "debug": false
}
[2022-10-31 19:00:29,805] [WARNING] [runner.py:179:fetch_hostfile] Unable to find hostfile, will proceed with training with local resources only.
[2022-10-31 19:00:29,837] [INFO] [runner.py:507:main] cmd = /usr/bin/python3 -u -m deepspeed.launcher.launch --world_info=eyJsb2NhbGhvc3QiOiBbMF19 --master_addr=127.0.0.1 --master_port=29500 train.py --deepspeed_config settings/ds_config_zero.json --data ETTh1 --seq_len 48 --pred_len 24 --dec_seq_len 48 --hidden_size 96 --n_encoder_layers 3 --n_decoder_layers 3 --encoder_attention query_selector_0.85 --decoder_attention full --n_head